In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Cash ladder

Demonstration of how to compute a cash ladder for a portfolio.

Attributes
----------
bucketed cashflow
cash
cash ladder
"""

toggle_code("Toggle Docstring")

# Cash Ladder

In this notebook, we demonstrate how users can create a cash ladder with LUSID. For the purposes of this notebook, we will first upsert a portfolio and some example transactions. Then we will use bucketed cashflows to create a cash ladder. This will help us understand how those transactions have affected our balance over time.

## Setup LUSID

We start by importing relevant libraries, authenticating our user, and creating our API client.

In [2]:
# Import general purpose packages
import os
import json
import pandas as pd
import numpy as np
import pytz
import warnings
from datetime import datetime, timedelta
from IPython.display import display

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.seed_sample_data import seed_data

# Configure notebook warnings and pandas display
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

# Define the relevant APIs we will use
transaction_portfolio_api = api_factory.build(
    lusid.api.TransactionPortfoliosApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)

## 1. Prepare & Setup Data 

In this notebook, we have a portfolio called EQUITY_FUND.

In [3]:
# Load a mapping file for loading data
with open("config/cash_ladder_seed_data.json", "r") as seed_data_config:
    mapping_file = json.loads(seed_data_config.read())

# Portfolio scope and code
scope = "cash-ladder-001"
portfolio_code = "EQUITY_FUND"

# Load a file of equity transactions
transactions_file = r"data/cash_ladder.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code

# Load portfolios, instruments, and transactions
seed_data_response = seed_data(
    api_factory=api_factory,
    domains=["portfolios", "instruments", "transactions"],
    scope=scope,
    transaction_file=transactions_file,
    file_type="csv",
    mappings=mapping_file
)

## 2. Upload Quote Data

We will load in some quotes for our equity instruments.

In [4]:
equity_prices = pd.read_csv("data/equity_quotes.csv")
equity_prices.head(n=5)

,instrument_id,instrument_name,date,price,currency
0,EQ_GB001,vodafone,29/03/2020,114.60,GBP
1,EQ_GB001,vodafone,30/03/2020,114.50,GBP
2,EQ_GB001,vodafone,31/03/2020,118.54,GBP
3,EQ_GB001,vodafone,01/04/2020,117.64,GBP
4,EQ_GB001,vodafone,02/04/2020,117.46,GBP


We will then upsert these quotes to LUSID.

In [5]:
# Iterate through our dataframe and add each quote to an UpsertQuoteRequest
instrument_quotes = {
    index: models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider="Lusid",
                instrument_id=row["instrument_id"],
                instrument_id_type="ClientInternal",
                quote_type="Price",
                field="mid",
            ),
            effective_at=str(datetime.strptime(
                row['date'], '%d/%m/%Y').date()),
        ),
        metric_value=models.MetricValue(
            value=row['price'], unit=row['currency']),
        scale_factor=100,
    )
    for index, row in equity_prices.iterrows()
}

# Upsert the quotes
response = quotes_api.upsert_quotes(
    scope=scope, request_body=instrument_quotes)

if response.failed == {}:
    print(
        f"Quotes successfully loaded into LUSID. {len(response.values)} quotes loaded.")
else:
    print(
        f"Some failures occurred during quote upsertion, {len(response.failed)} did not get loaded into LUSID.")

Quotes successfully loaded into LUSID. 39 quotes loaded.


## 3. Upsert Configuration Recipe

Create and upsert basic configuration recipe to be used for valuation.

In [6]:
# Create a configuration recipe
configuration_recipe = models.ConfigurationRecipe(
    scope=scope,
    code=portfolio_code,

    # Define the market rules and which quotes to use
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Quote.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D.0D",
            )
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        )
    )
)

# Upsert the recipe to LUSID
config_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

## 4. Produce Cash Ladder

### 4.1 Get Bucketed Cash Flows

We will use the `GetBucketedCashFlows` API endpoint to acquire the data for our cash ladder. This aggregates a transaction portfolio's instruments by date.

In [7]:
# Set the star and end dates for our cash ladder
start_date = datetime(year=2020, month=3, day=25, tzinfo=pytz.UTC)
end_date = datetime(year=2020, month=4, day=10, tzinfo=pytz.UTC)

# Create a string with the current time
time_now = datetime.now().strftime("%H:%M:%S")

# Create a request to get the bucketed cashflows
bucket_response = transaction_portfolio_api.get_bucketed_cash_flows(
    scope=scope,
    code=portfolio_code,

    bucketed_cash_flow_request=models.BucketedCashFlowRequest(
        # When bucketing, there is not a unique way to allocate the bucket points. Supported options are: [RoundDown, RoundUp]
        # See API documentation for reference: https://www.lusid.com/docs/api/#operation/GetBucketedCashFlows
        rounding_method="RoundDown",

        # List every day between our start and end dates
        bucketing_dates=[
            start_date + timedelta(days=x) for x in range((end_date - start_date).days)
        ],

        # Set the valuation recipe
        recipe_id=models.ResourceId(
            scope=scope,
            code=portfolio_code,
        ),

        # Group by date and currency
        group_by=["Valuation/CashFlowDate/RoundDown",
                  "Valuation/CashFlowCurrency"],

        # Add several address for more information
        addresses=[
            "Valuation/CashFlowAmount", "Valuation/OpenBalance", "Valuation/CloseBalance", "Valuation/ActivityType", "Valuation/Movements"
        ],

        # Set effective date, cash flow type, and start window
        effective_at=end_date,
        cash_flow_type="PortfolioCashFlow",
        window_start=datetime(1, 1, 2, tzinfo=pytz.UTC).isoformat(),
    )
)

### 4.2 Build Cash Ladder

Now we have our bucketed cashflows, we will extract the cash impacts and create our cash ladder.

#### Data Preprocessing

First we need to extract the relevant information from the response.

In [8]:
# Mapping function used to extract value and unit pairs from a 0D column
def format_result0D_column(values: pd.Series):
    return [(x['value'], x["units"]) for x in values]


# Convert the response to a pandas dataframe
data = pd.DataFrame.from_records(np.array(bucket_response.data))

# Use the mapping function to format CashFlowAmount
data["Append(Valuation/CashFlowAmount)"] = data["Append(Valuation/CashFlowAmount)"].map(format_result0D_column)

# Group rows by currency
data = data.groupby(["Valuation/CashFlowCurrency"], sort=False)

# Define function to convert a dataframe row to a record


def to_record(row):

    # We make use of the CashLadderRecord model
    return models.CashLadderRecord(
        effective_date=pd.to_datetime(row["Valuation/CashFlowDate/RoundDown"]),
        open=row["Valuation/OpenBalance"],
        close=row["Valuation/CloseBalance"],
        activities=list(zip(row["Append(Valuation/Movements)"],
                        row["Append(Valuation/CashFlowAmount)"])),
    )

#### Constructing Cash Ladders

Now we can construct and view the cash ladder for each currency with available quote data. In this case, we have two available: GBP and USD.

In [9]:
# Iterate through each dataframe group (i.e. currency)
for (key, df) in data:

    currency = key[0]
    print(f"Cash ladder for currency {currency}:")

    # Create a new dataframe for each cash ladder
    ladder = pd.DataFrame(columns=["ReportRunTime", "SettlementDate",
                          "Currency", "Portfolio", "CashLadderStatus", "CashImpact"])

    # Iterate through each cash row in our dataframe
    for _, row in df.iterrows():
        if row["Valuation/ActivityType"] != "Activity":
            continue
        # Convert the row to a CashLadderRecord
        record = to_record(row)

        # Create a new row for our SOD balance
        ladder.loc[len(ladder)] = {
            "ReportRunTime": time_now,
            "SettlementDate": record.effective_date.strftime("%Y-%m-%d"),
            "Currency": currency,
            "Portfolio": portfolio_code,
            "CashLadderStatus": "Open Balance",
            "CashImpact": record.open
        }

        # Iterate through each activity
        for item in record.activities:
            if item[0] != "Closed" and item[0] != "Open":

                # Create a new row for each cash impact
                ladder.loc[len(ladder)] = {
                    "ReportRunTime": time_now,
                    "SettlementDate": record.effective_date.strftime("%Y-%m-%d"),
                    "Currency": item[1][1][1:4],
                    "Portfolio": portfolio_code,
                    "CashLadderStatus": f"Cash impact from {item[0]}",
                    "CashImpact": item[1][0]
                }

        # Create a new row for our EOD balance
        ladder.loc[len(ladder)] = {
            "ReportRunTime": time_now,
            "SettlementDate": record.effective_date.strftime("%Y-%m-%d"),
            "Currency": currency,
            "Portfolio": portfolio_code,
            "CashLadderStatus": "Closed Balance",
            "CashImpact": record.close
        }

    # Display the cash ladder for this currency
    display(ladder.set_index(['SettlementDate', 'Portfolio']))

Cash ladder for currency USD:


ReportRunTime Currency        CashLadderStatus  \
SettlementDate Portfolio                                                    
2020-03-30     EQUITY_FUND      09:38:33      USD            Open Balance   
               EQUITY_FUND      09:38:33      USD  Cash impact from Side1   
               EQUITY_FUND      09:38:33      USD          Closed Balance   
2020-04-01     EQUITY_FUND      09:38:33      USD            Open Balance   
               EQUITY_FUND      09:38:33      USD  Cash impact from Side2   
               EQUITY_FUND      09:38:33      USD          Closed Balance   
2020-04-03     EQUITY_FUND      09:38:33      USD            Open Balance   
               EQUITY_FUND      09:38:33      USD  Cash impact from Side2   
               EQUITY_FUND      09:38:33      USD  Cash impact from Side1   
               EQUITY_FUND      09:38:33      USD          Closed Balance   
2020-04-04     EQUITY_FUND      09:38:33      USD            Open Balance   
               EQUITY_FUND      09:38:33      USD  Cash impact from Side2   
               EQUITY_FUND      09:38:33      USD          Closed Balance   

                            CashImpact  
SettlementDate Portfolio                
2020-03-30     EQUITY_FUND         0.0  
               EQUITY_FUND  20000000.0  
               EQUITY_FUND  20000000.0  
2020-04-01     EQUITY_FUND  20000000.0  
               EQUITY_FUND  -3000000.0  
               EQUITY_FUND  17000000.0  
2020-04-03     EQUITY_FUND  17000000.0  
               EQUITY_FUND         0.0  
               EQUITY_FUND  -2000000.0  
               EQUITY_FUND  15000000.0  
2020-04-04     EQUITY_FUND  15000000.0  
               EQUITY_FUND  -3000000.0  
               EQUITY_FUND  12000000.0

Cash ladder for currency GBP:


ReportRunTime Currency        CashLadderStatus  \
SettlementDate Portfolio                                                    
2020-03-31     EQUITY_FUND      09:38:33      GBP            Open Balance   
               EQUITY_FUND      09:38:33      GBP  Cash impact from Side1   
               EQUITY_FUND      09:38:33      GBP          Closed Balance   
2020-04-01     EQUITY_FUND      09:38:33      GBP            Open Balance   
               EQUITY_FUND      09:38:33      GBP  Cash impact from Side2   
               EQUITY_FUND      09:38:33      GBP  Cash impact from Side1   
               EQUITY_FUND      09:38:33      GBP          Closed Balance   
2020-04-03     EQUITY_FUND      09:38:33      GBP            Open Balance   
               EQUITY_FUND      09:38:33      GBP  Cash impact from Side2   
               EQUITY_FUND      09:38:33      GBP  Cash impact from Side1   
               EQUITY_FUND      09:38:33      GBP          Closed Balance   
2020-04-04     EQUITY_FUND      09:38:33      GBP            Open Balance   
               EQUITY_FUND      09:38:33      GBP  Cash impact from Side1   
               EQUITY_FUND      09:38:33      GBP          Closed Balance   
2020-04-05     EQUITY_FUND      09:38:33      GBP            Open Balance   
               EQUITY_FUND      09:38:33      GBP  Cash impact from Side1   
               EQUITY_FUND      09:38:33      GBP          Closed Balance   
2020-04-06     EQUITY_FUND      09:38:33      GBP            Open Balance   
               EQUITY_FUND      09:38:33      GBP  Cash impact from Side1   
               EQUITY_FUND      09:38:33      GBP          Closed Balance   

                            CashImpact  
SettlementDate Portfolio                
2020-03-31     EQUITY_FUND         0.0  
               EQUITY_FUND    500000.0  
               EQUITY_FUND    500000.0  
2020-04-01     EQUITY_FUND    500000.0  
               EQUITY_FUND   -200000.0  
               EQUITY_FUND  10000000.0  
               EQUITY_FUND  10300000.0  
2020-04-03     EQUITY_FUND  10300000.0  
               EQUITY_FUND  -6900000.0  
               EQUITY_FUND  -1000000.0  
               EQUITY_FUND   2400000.0  
2020-04-04     EQUITY_FUND   2400000.0  
               EQUITY_FUND   1000000.0  
               EQUITY_FUND   3400000.0  
2020-04-05     EQUITY_FUND   3400000.0  
               EQUITY_FUND   3000000.0  
               EQUITY_FUND   6400000.0  
2020-04-06     EQUITY_FUND   6400000.0  
               EQUITY_FUND  20000000.0  
               EQUITY_FUND  26400000.0